## 示例：分组加权平均数和相关系数

根据groupby的“拆分－应用－合并”范式，可以进行DataFrame的列与列之间或两个Series之间的运算（比如分组加权平均）。以下面这个数据集为例，它含有分组键、值以及一些权重值：

In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a','b', 'b', 'b', 'b'],
                   'data': np.random.randn(8),
                   'weights': np.random.rand(8)})


df

,category,data,weights
0,a,1.407320,0.490298
1,a,-0.395946,0.846310
2,a,-0.899899,0.580343
3,a,-1.230533,0.549366
4,b,-0.878602,0.354319
5,b,-0.669217,0.465249
6,b,-0.148792,0.214472
7,b,-0.834052,0.829824


然后可以利用category计算分组加权平均数：

In [6]:
grouped = df.groupby('category')

get_wavg = lambda g: np.average(g['data'], weights=g['weights'])

In [7]:
grouped.apply(get_wavg)

category
a   -0.341947
b   -0.722524
dtype: float64

另一个例子，考虑一个来自Yahoo!Finance的数据集，其中含有几只股票和标准普尔500指数（符号SPX）的收盘价：

In [8]:
close_px = pd.read_csv('../../examples/stock_px.csv', 
                       parse_dates=True,
                       index_col=0)



In [9]:
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [10]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


来做一个比较有趣的任务：计算一个由日收益率（通过百分数变化计算）与SPX之间的年度相关系数组成的DataFrame。下面是一个实现办法，我们先创建一个函数，用它计算每列和SPX列的成对相关系数：

In [11]:
spx_corr = lambda x: x.corrwith(x['SPX'])

接下来，我们使用pct_change计算close_px的百分比变化：

In [12]:
rets = close_px.pct_change().dropna()

最后，我们用年对百分比变化进行分组，可以用一个一行的函数，从每行的标签返回每个datetime标签的year属性：

In [13]:
get_year = lambda x: x.year

In [16]:
by_year = rets.groupby(get_year)
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


当然，你还可以计算列与列之间的相关系数。这里，我们计算Apple和Microsoft的年相关系数：

In [17]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64